# Netflix Movies and TV Shows 에 대한 데이터를 가지고 EDA 진행하였습니다.

## [Eng sub] This time, I conducted EDA with Netflix Movies and TV Shows.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [ ]:
netflix = pd.read_csv("../input/netflix-shows/netflix_titles.csv")

netflix.tail(5)

**기본적인 데이터의 분석입니다.**

**Basic analysis of data**

In [ ]:
netflix.count()

In [ ]:
netflix.info()

In [ ]:
netflix.nunique()

# 1. Ratio of Netflix movies and TV shows

**아래 두 개의 plot은 넷플릭스의 영화와 TV Show 비율에 대한 plot입니다.**

**[Eng sub] Two plots for Netflix's movie and TV Show ratio.**

In [ ]:
sns.set(style="darkgrid")
ax = sns.countplot(x="type", data=netflix)

In [ ]:
fig = px.pie(netflix['type'].value_counts().reset_index(), values='type', names='index', template='plotly_dark')
fig.show()

**넷플릭스에는 영화가 약 68.4%, TV Show가 31.6%의 비율로 등록되어 있습니다. 이는 약 2:1의 비율로 이루어져 있습니다.**

**Netflix has registered about 68.4 percent for movies and 31.6 percent for TV Show. It consists of a ratio of about 2:1.**

In [ ]:
netflix_shows=netflix[netflix['type']=='TV Show']
netflix_movies=netflix[netflix['type']=='Movie']

# 2. 넷플릭스 영화 및 TV쇼가 언제 개봉했는지, 혹은 언제 공개되었는지에 대한 분석입니다.

## 2. Analysis of when Netflix movies and TV shows were released.

In [ ]:
fig = px.histogram(netflix, x = 'release_year', color='type', marginal="box", template="plotly_dark")
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.set(style="darkgrid")
sns.countplot(ax = ax, data = netflix, x = 'release_year', hue="type")
plt.legend(loc='center')
plt.xticks(rotation='70')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
ax = sns.countplot(y="release_year", data=netflix_movies, order=netflix_movies['release_year'].value_counts().index[0:15])

**넷플릭스 영화에 대한 데이터는 2017년이 680개가량으로 가장 많습니다.**

**The data on Netflix movies was the highest in 2017 with about 680 films.**

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
ax = sns.countplot(y="release_year", data=netflix_shows, order=netflix_shows['release_year'].value_counts().index[0:15])

**넷플릭스 티비쇼에 대한 데이터는 2019년이 450개가량으로 가장 많습니다.**

**The largest amount of data on Netflix TV shows is around 450 in 2019.**

#### 이것으로 보아, 최근 넷플릭스는 영화보다 TV Show에 선호도를 강하게 보이는 것으로 생각됩니다.

#### From this point of view, Netflix seems to have a stronger preference for TV shows than movies these days.

# 3. Rating, 연령분석

**다음으로, 영화의 rating에 대한 분석을 시작하겠습니다.**

**Next, I will start the analysis of the rating of the movie.**

In [ ]:
fig = px.histogram(
  netflix,
  x = 'rating',
  color = 'type',
  marginal = 'box',
  template = 'plotly_dark'
)

fig.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
ax = sns.countplot(x="rating", data=netflix, order=netflix['rating'].value_counts().index[0:15])

**TV-MA 와 TV-14 등급이 가장 많은 것으로 보이지만, 이는 모두에게 친숙하지 않기 때문에 연령으로 바꿔 표현하겠습니다.**

**TV-MA and TV-14 appear to have the highest ratings, but this is not familiar to everyone, so I'll change it to age.**

#### 아마존의 심의등급은 아래와 같습니다.

#### Amazon's review rating is as follows.

In [ ]:
ratings_data = { 'TV-PG': 7,'TV-MA': 18,'TV-Y7-FV': 7,'TV-Y7': 7,
            'TV-14': 16,'R': 18,'TV-Y': 0,'NR': 18,'PG-13': 13,
            'TV-G': 0,'PG': 7,'G': 0,'UR': 18,'NC-17': 18}

ratings = pd.Series(ratings_data, name='recommand_age')
ratings

In [ ]:
netflix = pd.merge(netflix, ratings, left_on='rating', right_index=True)
netflix_shows = netflix[netflix['type']=='TV Show']
netflix_movies = netflix[netflix['type']=='Movie']

rating_df = netflix.groupby(['rating', 'recommand_age']).agg({'show_id': 'count'}).reset_index()
rating_df.head()

**저는 레이팅을 적당한 연령으로 바꿔 합병하고, 이를 플롯으로 표현하였습니다.**

I changed the rating to the appropriate age and merged it and expressed it as a plot.

**위와 같이 원래의 Rating, 치환된 Recommand_age, Count의 세가지 컬럼으로 구성된 것을 보실 수 있습니다.**

As shown above, you can see that there are three columns: Original Ratings, Replacement Command_age, and Count.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
    go.Pie(labels=netflix_shows['recommand_age'], values=netflix_shows['show_id']),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=netflix_movies['recommand_age'], values=netflix_movies['show_id']),
    row=1, col=2
)

fig.update_layout(template='plotly_dark',
    annotations=[dict(text='Movies', x=0.22, y=1, font_size=16),
                 dict(text='TV Shows', x=0.77, y=1, font_size=16)])

fig.show()

In [ ]:
data_by_age = netflix.groupby(["recommand_age",'type'])['show_id'].count()

data_by_age.plot.pie(figsize=(15,10))
plt.show()

**18세 이상의 영화가 가장 비율이 높음을 알 수 있습니다. 또한, 16세 이상의 영화 및 TV쇼가 두 분야 모두 75%가량 되는 것을 확인하였습니다.**

**You can see that movies over 18 have the highest proportion. Also, we found that there are about 75% of movie and TV shows over the age of 16 in both areas.**

# 4. 국가별 분석

## 4. Regional Analysis

In [ ]:
import collections as c

country_count = c.Counter(", ".join(netflix['country'].dropna()).split(", "))
top_ten_countries = country_count.most_common()
rank = {}
for x in top_ten_countries[::-1]:
  rank[x[0]] = x[1]
rank_series = pd.DataFrame(data=rank, index=['count']).T

In [ ]:
rank_series

In [ ]:
data_df = pd.read_json("../input/countries-to-alpha3/countries.json")
rank_series = pd.merge(data_df, rank_series, left_on='name', right_index=True)
rank_series

In [ ]:
fig = px.scatter_geo(rank_series, locations= 'code',
                     hover_name = 'name', size="count", color='count', color_continuous_scale="icefire",range_color=(0, 200),
                     projection="natural earth", size_max=80, template="plotly_dark")
fig.show()

### countries.json 파일을 사용하여 나라이름을 국가코드로 변경한 뒤, 이를 Bubble map으로 표현하였습니다.

### I used the country.json file to change the country name to the country code and expressed it as Bubble map.

# 5. 상영시간 분석

## 5. Duration

In [ ]:
netflix_movies['duration']=netflix_movies['duration'].str.replace(' min','')
netflix_movies['duration']=netflix_movies['duration'].astype(str).astype(int)
netflix_movies['duration']

In [ ]:
plt.figure(figsize=(12,10))
plt.style.use("dark_background")
sns.kdeplot(data=netflix_movies['duration'], shade=True)
plt.show()

**kde 플롯을 통해 살펴보았을 때, 100분 근처의 영화가 가장 많은 비율을 차지하고 있음을 알 수 있습니다.**

**When you look at the kde plot, you can see that the movies that are close to 100 minutes duration account for the largest proportion.**

# 6. 장르 분석

## 6. Genres

In [ ]:
from collections import Counter

genres=list(netflix['listed_in'])
gen=[]

for i in genres:
    i=list(i.split(','))
    for j in i:
        gen.append(j.replace(' ',""))
g=Counter(gen)

In [ ]:
genres_df = pd.DataFrame.from_records(g.most_common(), columns=['Movie&TV','number'])

genres_df.head()

In [ ]:
fig = px.bar(genres_df, x='Movie&TV', y='number', template="plotly_dark", color="number")
fig.show()

In [ ]:
genres_df = pd.DataFrame.from_records(g.most_common(), columns=['Movie&TV','number'])

plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
sns.barplot(data=genres_df, x="Movie&TV", y="number")
plt.xticks(rotation=90)
plt.show()

# 7. date_added 컬럼을 가지고 영화의 등록날짜 분석을 해보겠습니다.

## 7. Let's analyze the registration date of the movie with the date_added column.

In [ ]:
netflix_date = netflix_shows[['date_added']].dropna()
netflix_date['year'] = netflix_date['date_added'].apply(lambda x : x.split(', ')[-1])
netflix_date['month'] = netflix_date['date_added'].apply(lambda x : x.lstrip().split(' ')[0])
netflix_date['day'] = netflix_date['date_added'].apply(lambda x : x.lstrip().split(' ')[1])
netflix_date['day'] = netflix_date['day'].str.replace(',','')

month2int = {
    'None' : 0,
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

netflix_date['month'] = netflix_date['month'].map(month2int)
netflix_date_to_time = pd.to_datetime(netflix_date[['year', 'month', 'day']])

netflix_date_to_time.name = 'datetime'
netflix_date = pd.concat([netflix_date, netflix_date_to_time], axis=1)

netflix_date

In [ ]:
import collections as c

g=Counter(netflix_date['datetime'])
time = pd.DataFrame.from_records(list(g.items()), columns=['datetime','number'])

time

In [ ]:
px.bar(time, x='datetime', y='number', template='plotly_dark', color='number')

In [ ]:
netflix_count = netflix_date_to_time.value_counts().sort_index()

netflix_cumsum = netflix_count.cumsum()

px.bar(netflix_cumsum, template='plotly_dark')

**잘 보이지 않지만 드래그하여 확대해서 보면 더 자세한 데이터를 볼 수 있습니다. 약 2016-2020년 데이터를 확대하여 보시기를 추천드립니다.**

**You can't see very well, but you can drag and zoom to see more detailed data. I recommend you to zoom in on the 2016-2020 data.**

**등록날짜 역시 넷플릭스의 성장세에 따라 최근영화가 많이 등록되고 있음을 알 수 있습니다.**

**The registration date also shows that a lot of recent movies are being registered as Netflix grows.**